In [1]:
import os

os.environ["COHERE_API_KEY"] = 'E2efronXbGqXSR0sclyInSjXptFOjdIjsbCeXFwP'

from langchain_cohere import ChatCohere
from langchain_ollama import ChatOllama

In [2]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate

In [3]:
from pydantic import BaseModel, Field

In [4]:
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
XL_PATH = r"../dataset/screening/articles_to_screen.csv"
OUT_DIR = r"../dataset/screening/ai_decision.csv"

: 

In [ ]:
# screened abstracts

df = pd.read_csv(XL_PATH)[["key", "title", "abstract"]]

display(df.head())
print(len(df))

In [ ]:
class OutputSchema(BaseModel):
    thoughts: str = Field(description= "thoughts of the model")
    decision: int = Field(description="1 if the abstract was selected, 0 otherwise")
    reason: str = Field(description = "generate a consise one sentence long reason for the decision")


llm = ChatCohere(model="command-r", temperature=0.0)
# llm = ChatOllama(model="deepseek-r1:32b", temperature=0.0)
struct_llm = llm.with_structured_output(OutputSchema)

In [ ]:
# messages = [
#     ('system',  "You are a helpful AI agent that assists in accurately screening the article using its abstract. " 
#                 "Based solely on the abstract provided, determine whether the article discusses an APPLICATION OF AI METHODS IN CARBON ION THERAPY? "
#                 "Your decision should be '0' for NO or '1' for YES. Then, generate a concise, one-sentence reason for your decision."
#     ),
#     ('human', "abstract:\n\n title: {title}, \n content: {abstract}")
#   ]

messages = [
    ('system',  "You are a helpful AI assistant that accurately screens and SELECTS articles relevant to given TOPIC, for inclusion in a literature review, based solely on their ABSTRACT. " 
                "Your decision should be '1' for YES or '0' otherwise. Then, generate a concise, one-sentence reason for your decision."
    ),
    ('human', "TOPIC: APPLICATIONS OF AI METHODS IN CARBON ION THERAPY\n\nABSTRACT:\n\n title: {title}, \n content: {abstract}")
  ]

prompt_template = ChatPromptTemplate.from_messages(messages)
chain = prompt_template | struct_llm

decision_df = {"key":[], "title":[], "abstract":[], "decision":[], "reason":[], "thoughts":[]}

rows = [row.to_dict() for i,row in df.iterrows()][:5]

for row in tqdm(rows):
  # key, title, selection = row.to_dict()["key"], row.to_dict()["title"], row.to_dict()["selection"]

  inputs = {key:row[key] for key in ["title", "abstract"]} 

  output = chain.invoke(inputs)

  for key,val in {**row, **dict(output)}.items():
      decision_df[key].append(val)
  
decision_df = pd.DataFrame(decision_df)
if not os.path.exists(OUT_DIR):
   os.makedirs(OUT_DIR)
decision_df.to_csv(os.path.join(OUT_DIR, "ai_decision.csv"), index=False)


In [ ]:
decision_df

In [ ]:
# generating summaries

In [ ]:
# this doesn't work very well
# from langchain_community.document_loaders import PyPDFLoader


# file_path = r"1-s2.0-S1120179724002163-main.pdf"
# loader = PyPDFLoader(file_path, mode='single')
# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

# manuscript = "\n".join([page.page_content for page in pages])

In [ ]:
# import pymupdf4llm

# file_path = r"1-s2.0-S1120179724002163-main.pdf"
# manuscript = pymupdf4llm.to_markdown(file_path)

# ref_index = manuscript.lower().find("references")

# # Keep everything up to and including "References"
# if ref_index != -1:
#     manuscript = manuscript[:ref_index + len("references")]

# # Now md_text contains only up to the References section
# print(manuscript)

In [ ]:


# docs = []
# for doc in docs_lazy:
#     docs.append(doc)

# manuscript = docs[0].page_content


In [ ]:

# class OutputSchema(BaseModel):
#     thoughts: str = Field(description= "thoughts of the model")
#     study_objective: str = Field(description="The objective of the article")
#     ai_methods: str = Field(description="The AI method(s) used in the article in the context of carbon ion therapy. Describe each AI method at least in one sentence")
#     application_domain: str = Field(description="the specific carbon ion therapy application domain")
#     dataset_characteristics: str = Field(description="characteristics of the dataset, including the training, validation, test split; also external validation information, if present")
#     key_findings: str = Field(description="key findings mentioned in the article")
#     pros_and_cons: str = Field(description="strengths and limitations of the study")
#     future_direction: str = Field(description="future directions (if stated)")

# llm = ChatCohere(model='command-r', temperature=0.0)
# # llm = ChatOllama(model="deepseek-r1:32b", temperature=0.0)
# struct_llm = llm.with_structured_output(OutputSchema)

# messages = [
#     ('system',  "You are a helpful AI agent that assists in accurately summarizing the article in a standardized format "
#                 "to be used for the literature review titled 'APPLICATIONS OF AI IN CARBON ION THERAPY'. \n"
#                 "Summarize the article under the following standard headers:\n"
#                 "'1. Study objective', '2. AI methods used', '3. carbon therapy application domain', '4. Dataset characteristics', '5. Key findings', "
#                 "'6. Strengths and limitations', '7. Future directions (if stated)' "
#     ),
#     ('human', "article: \n {article}")
#   ]

# prompt_template = ChatPromptTemplate.from_messages(messages)
# chain = prompt_template | struct_llm


In [ ]:

class OutputSchema(BaseModel):
    thoughts: str = Field(description= "thoughts of the model")
    Q1_answer: str = Field(description="answer to Q1.What is the aim of this study?")
    Q2_answer: str = Field(description="answer to Q2. Which of the following categories does this article belong to: treatment planning, tumor control prediction, normal tissue complication prediction, or generative AI")
    Q3_answer: str = Field(description="answer to Q3. Describe the dataset and provide the training, validation and test strategy involved in AI modelling, including the corresponding sample counts. ")
    Q4_answer: str = Field(description="answer to Q4. Define the AI methodology used by the authors in this category")
    Q5_answer: str = Field(description="answer to Q5. What are the strengths and weaknesses of the methodology followed? ")
    Q6_answer: str = Field(description="answer to Q6. Can you summarize the results in terms of the performance metrics and check how appropriate the metrics were to evaluate the AI model? ")
    Q7_answer: str = Field(description="answer to Q7. What are the strong and weak arguments that the authors are pointing out in the discussion")
    Q8_answer: str = Field(description="answer to Q8. What is their conclusion, and what are their main arguments to support it? ")
    Q9_answer: str = Field(description="answer to Q9. Can you generate a critical summary combining all this information in 300 characters?")

# llm = ChatCohere(model='command-r', temperature=0.0)
llm = ChatOllama(model="deepseek-r1:32b", temperature=0.0)
struct_llm = llm.with_structured_output(OutputSchema)

In [ ]:
messages = [
    ('system', '''You are an expert AI agent that can perform concise yet accurate review on the input ARTICLE excluding the abstract by answering the following questions: Q1-Q9 
                Q1. What is the aim of this study? 
                Q2. Which of the following categories does this article belong to: treatment planning, tumor control prediction, normal tissue complication prediction, or generative AI 
                Q3. Describe the dataset and provide the training, validation and test strategy involved in AI modelling, including the corresponding sample counts. 
                Q4. Define the AI methodology used by the authors in this category 
                Q5. What are the strengths and weaknesses of the methodology followed? 
                Q6. Can you summarize the results in terms of the performance metrics and check how appropriate the metrics were to evaluate the AI model? 
                Q7. What are the strong and weak arguments that the authors are pointing out in the discussion 
                Q8. What is their conclusion, and what are their main arguments to support it? 
                Q9. Can you generate a critical summary combining all this information in 300 characters? '''
     ),
     ('human', "ARTICLE: \n {article}")
]

prompt_template = ChatPromptTemplate.from_messages(messages)
chain = prompt_template | struct_llm

In [ ]:
from langchain_pymupdf4llm import PyMuPDF4LLMLoader

file_path = r"1-s2.0-S1120179724002163-main.pdf"
loader = PyMuPDF4LLMLoader(file_path, mode='single') #we are loading all the pages of the documents in a single page
docs = loader.load()
manuscript = docs[0].page_content
output = chain.invoke({"article":manuscript})

In [ ]:
output